In [1]:
import os
dataset = 'Dataset/ChildData/Croped/'
print(os.listdir(dataset))

classes = [folder for folder in os.listdir(dataset) if os.path.isdir(os.path.join(dataset, folder))]
count = []

for folder in classes:
    folder_path = os.path.join(dataset, folder)
    num_image = len([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])
    count.append(num_image)
    print(f"{folder} = {num_image}")


['Fear', 'Sad', 'Angry', 'Test', 'Neutral', 'unknow expression', 'Surprised', 'Happiness', '.ipynb_checkpoints', 'Train', 'Disgust']
Fear = 33
Sad = 70
Angry = 53
Test = 0
Neutral = 54
unknow expression = 8
Surprised = 91
Happiness = 98
.ipynb_checkpoints = 0
Train = 0
Disgust = 43


In [2]:
classes_needs = ['Fear', 'Sad', 'Angry', 'Neutral', 'Surprised', 'Disgust', 'Sad']

In [3]:
import os
dataset = 'Dataset/ChildData/Croped/Train'
print(os.listdir(dataset))

['Fear', 'Sad', 'Angry', 'Neutral', 'Surprised', 'Happiness', 'Disgust']


In [4]:
classes = [folder for folder in os.listdir(dataset) if os.path.isdir(os.path.join(dataset, folder))]
count = []

for folder in classes:
    folder_path = os.path.join(dataset, folder)
    num_image = len([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])
    count.append(num_image)
    print(f"{folder} = {num_image}")


Fear = 33
Sad = 65
Angry = 42
Neutral = 50
Surprised = 72
Happiness = 78
Disgust = 42


In [5]:
import os
dataset = 'Dataset/ChildData/Croped/Train/'
print(os.listdir(dataset))

classes = [folder for folder in os.listdir(dataset) if os.path.isdir(os.path.join(dataset, folder))]
count = []

for folder in classes:
    folder_path = os.path.join(dataset, folder)
    num_image = len([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])
    count.append(num_image)
    print(f"{folder} = {num_image}")

['Fear', 'Sad', 'Angry', 'Neutral', 'Surprised', 'Happiness', 'Disgust']
Fear = 33
Sad = 65
Angry = 42
Neutral = 50
Surprised = 72
Happiness = 78
Disgust = 42


In [6]:
import os
dataset = 'Dataset/ChildData/Croped/Test/'
print(os.listdir(dataset))

classes = [folder for folder in os.listdir(dataset) if os.path.isdir(os.path.join(dataset, folder))]
count = []

for folder in classes:
    folder_path = os.path.join(dataset, folder)
    num_image = len([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])
    count.append(num_image)
    print(f"{folder} = {num_image}")

['Fear', 'Sad', 'Angry', 'Neutral', 'Surprised', 'Happiness', 'Disgust']
Fear = 14
Sad = 23
Angry = 11
Neutral = 18
Surprised = 19
Happiness = 20
Disgust = 17


In [7]:
import os
import glob
import torch
import torch.nn as nn
import torchvision.transforms as tt
import torchvision
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.utils.tensorboard import SummaryWriter
import pathlib

2025-04-09 14:27:49.242661: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-09 14:27:49.481168: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-09 14:27:49.528649: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-09 14:27:49.713599: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-09 14:27:53.868807: W tensorflow/compiler/tf2

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [9]:
# # Define the data transformations
# train_tt = tt.Compose([
#     tt.RandomPerspective(distortion_scale=0.6, p=0.5),
#     tt.RandomRotation(degrees=(0, 180)),
#     tt.RandomAffine(degrees=(30, 70), translate=(0.1, 0.3), scale=(0.5, 0.75)),
#     tt.RandomAdjustSharpness(0, p=0.25),
#     tt.RandomAdjustSharpness(2, p=0.25),
#     tt.RandomAutocontrast(p=0.2),
#     tt.RandomEqualize(p=0.2),
#     tt.Resize((48, 48)),
#     tt.RandomHorizontalFlip(),
#     tt.ToTensor(),
#     tt.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
# ])

In [10]:
train_path = 'Dataset/ChildData/Croped/Train/'
test_path = 'Dataset/ChildData/Croped/Test/'

In [11]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from sklearn.preprocessing import LabelEncoder

class CustomDataset(Dataset):
    def __init__(self, data_path, transform=None):
        """
        Args:
            data_path (str): Path to the dataset (train or test).
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.data_path = data_path
        self.transform = transform
        
        # Get all class subfolders (e.g., 'Angry', 'Disgust', etc.)
        self.class_names = sorted(os.listdir(data_path))  # Assumes class names are folder names
        self.class_to_idx = {class_name: idx for idx, class_name in enumerate(self.class_names)}
        
        # Get image file paths along with corresponding labels
        self.image_paths = []
        self.labels = []
        
        for class_name in self.class_names:
            class_folder = os.path.join(data_path, class_name)
            if os.path.isdir(class_folder):
                for fname in os.listdir(class_folder):
                    if fname.endswith('.jpg'):
                        self.image_paths.append(os.path.join(class_folder, fname))
                        self.labels.append(self.class_to_idx[class_name])

    def __len__(self):
        # Return the number of images in the dataset
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        # Load an image
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        
        # Get the label (index of the class)
        label = self.labels[idx]
        
        # Apply the transformations if provided
        if self.transform:
            image = self.transform(image)
        
        return image, label

In [12]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to a fixed size (e.g., 224x224)
    transforms.ToTensor(),  # Convert image to a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize to pre-trained model standards
])

# Initialize dataset and dataloaders for training and testing
train_dataset = CustomDataset(data_path='Dataset/ChildData/Croped/Train/', transform=transform)
test_dataset = CustomDataset(data_path='Dataset/ChildData/Croped/Test/', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [13]:
next(iter(train_loader))[0].shape, next(iter(train_loader))[1].shape

(torch.Size([32, 3, 224, 224]), torch.Size([32]))

In [14]:

# train_loader = DataLoader(
#     torchvision.datasets.ImageFolder(train_path, transform=train_tt),
#     batch_size=32,
#     shuffle=True
# )

# test_loader = DataLoader(
#     torchvision.datasets.ImageFolder(test_path, transform=train_tt),
#     batch_size=32,
#     shuffle=False  # Test data should not be shuffled
# )

In [15]:
root = pathlib.Path(train_path)
classes = sorted([j.name.split('/')[-1] for j in root.iterdir()])
print(classes)

['Angry', 'Disgust', 'Fear', 'Happiness', 'Neutral', 'Sad', 'Surprised']


# Network Architecture

In [16]:
# class ConvNet(nn.Module):
#     def __init__(self, num_classes=7):
#         super(ConvNet, self).__init__()
#         self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=3, stride=1, padding=1)
#         self.bn1 = nn.BatchNorm2d(num_features=12)
#         self.relu1 = nn.ReLU()

        # self.pool = nn.MaxPool2d(kernel_size=2)

#         self.conv2 = nn.Conv2d(in_channels=12, out_channels=20, kernel_size=3, stride=1, padding=1)
#         self.relu2 = nn.ReLU()

#         self.conv3 = nn.Conv2d(in_channels=20, out_channels=32, kernel_size=3, stride=1, padding=1)
#         self.bn3 = nn.BatchNorm2d(num_features=32)
#         self.relu3 = nn.ReLU()

#         self.fc = nn.Linear(in_features=32 * 24 * 24, out_features=num_classes)

#     def forward(self, input):
#         output = self.conv1(input)
#         output = self.bn1(output)
#         output = self.relu1(output)
#         output = self.pool(output)

#         output = self.conv2(output)
#         output = self.relu2(output)

#         output = self.conv3(output)
#         output = self.bn3(output)
#         output = self.relu3(output)

#         output = output.view(-1, 32 * 24 * 24)
#         output = self.fc(output)
#         return output

In [17]:
# net = ConvNet(num_classes=7)
# model = net.to(device)

In [18]:
# optimizer = Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
# num_epochs = 50
# writer = SummaryWriter('runs/childnet_exp')
# criterion = nn.CrossEntropyLoss()

In [19]:
# # train_count = len(glob.glob(train_path + '/**/*.jpg'))
# test_count = len(glob.glob(test_path + '/**/*.jpg'))
# print(train_count, test_count)

In [20]:
# def train(epochs, train_loader, val_loader, criterion, optimizer, device, writer):
#     best_accuracy = 0
#     print("======================== Start Training ======================")
#     for e in range(epochs):
#         model.train()  # Set model to training mode
#         train_loss = 0
#         validation_loss = 0
#         train_correct = 0
#         val_correct = 0

#         # Train the model
#         for data, labels in train_loader:
#             data, labels = data.to(device), labels.to(device)
#             optimizer.zero_grad()
#             output = model(data)
#             loss = criterion(output, labels)
#             loss.backward()
#             optimizer.step()

#             train_loss += loss.item()
#             _, preds = torch.max(output, 1)
#             train_correct += torch.sum(preds == labels.data)

#         # Validate the model
#         model.eval()  # Set model to evaluation mode
#         with torch.no_grad():  # Disable gradient calculation
#             for data, labels in val_loader:
#                 data, labels = data.to(device), labels.to(device)
#                 val_outputs = model(data)
#                 val_loss = criterion(val_outputs, labels)
#                 validation_loss += val_loss.item()
#                 _, val_preds = torch.max(val_outputs, 1)
#                 val_correct += torch.sum(val_preds == labels.data)

#         train_loss = train_loss / train_count
#         train_acc = train_correct.double() / train_count
#         validation_loss = validation_loss / test_count
#         val_acc = val_correct.double() / test_count

#         # Log to TensorBoard
#         writer.add_scalar('Train Loss', train_loss, e)
#         writer.add_scalar('Train Accuracy', train_acc, e)
#         writer.add_scalar('Validation Loss', validation_loss, e)
#         writer.add_scalar('Validation Accuracy', val_acc, e)

#         print(f"Epoch {e+1}/{epochs}: "
#               f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc*100:.2f}%, "
#               f"Validation Loss: {validation_loss:.4f}, Validation Accuracy: {val_acc*100:.2f}%")

#         if val_acc > best_accuracy:
#             torch.save(model.state_dict(), 'best_checkpoint_test.model')
#             best_accuracy = val_acc

In [21]:
# train(num_epochs, train_loader, test_loader, criterion, optimizer, device, writer)

# Resnet50

In [22]:
# # Define the data transformations
# train_tt = tt.Compose([
#     tt.RandomPerspective(distortion_scale=0.6, p=0.5),
#     tt.RandomRotation(degrees=(0, 180)),
#     tt.RandomAffine(degrees=(30, 70), translate=(0.1, 0.3), scale=(0.5, 0.75)),
#     tt.RandomAdjustSharpness(0, p=0.25),
#     tt.RandomAdjustSharpness(2, p=0.25),
#     tt.RandomAutocontrast(p=0.2),
#     tt.RandomEqualize(p=0.2),
#     tt.Resize((224, 224)),  # ResNet-50 typically uses 224x224 image size
#     tt.RandomHorizontalFlip(),
#     tt.ToTensor(),
#     tt.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Standard ImageNet normalization
# ])

In [23]:
# train_path = 'Dataset/ChildNet/Dataset/Train/'
# test_path = 'Dataset/ChildNet/Dataset/Test/'

# train_loader = DataLoader(
#     torchvision.datasets.ImageFolder(train_path, transform=train_tt),
#     batch_size=32,
#     shuffle=True
# )

# test_loader = DataLoader(
#     torchvision.datasets.ImageFolder(test_path, transform=train_tt),
#     batch_size=32,
#     shuffle=False  # Test data should not be shuffled
# )

# root = pathlib.Path(train_path)
# classes = sorted([j.name.split('/')[-1] for j in root.iterdir()])
# print(classes)

In [24]:
# Load pre-trained ResNet-50 model
model = torchvision.models.resnet50(pretrained=True)

# Freeze early layers (optional, for transfer learning)
for param in model.parameters():
    param.requires_grad = False

# Replace the final fully connected layer to match the number of classes
model.fc = nn.Linear(model.fc.in_features, len(classes))

# Move model to the device (GPU or CPU)
model = model.to(device)

# Set optimizer for the final fully connected layer only (fine-tuning)
optimizer = Adam(model.fc.parameters(), lr=0.001, weight_decay=0.0001)
num_epochs = 100
writer = SummaryWriter('runs/childnet_resnet50')
criterion = nn.CrossEntropyLoss()  # can change this loss

train_count = len(glob.glob(train_path + '/**/*.jpg'))
test_count = len(glob.glob(test_path + '/**/*.jpg'))
print(train_count, test_count)

/raid/home/dgx1575/20je0167_anurag/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/raid/home/dgx1575/20je0167_anurag/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /raid/home/dgx1575/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 97.8M/97.8M [00:01<00:00, 70.0MB/s]


382 122


In [25]:
# def train(epochs, train_loader, val_loader, criterion, optimizer, device, writer):
#     best_accuracy = 0
#     print("======================== Start Training ======================")
#     for e in range(epochs):
#         model.train()  # Set model to training mode
#         train_loss = 0
#         validation_loss = 0
#         train_correct = 0
#         val_correct = 0

#         # Train the model
#         for data, labels in train_loader:
#             data, labels = data.to(device), labels.to(device)
#             optimizer.zero_grad()
#             output = model(data)
#             loss = criterion(output, labels)
#             loss.backward()
#             optimizer.step()

#             train_loss += loss.item()
#             _, preds = torch.max(output, 1)
#             train_correct += torch.sum(preds == labels.data)

#         # Validate the model
#         model.eval()  # Set model to evaluation mode
#         with torch.no_grad():  # Disable gradient calculation
#             for data, labels in val_loader:
#                 data, labels = data.to(device), labels.to(device)
#                 val_outputs = model(data)
#                 val_loss = criterion(val_outputs, labels)
#                 validation_loss += val_loss.item()
#                 _, val_preds = torch.max(val_outputs, 1)
#                 val_correct += torch.sum(val_preds == labels.data)

#         train_loss = train_loss / train_count
#         train_acc = train_correct.double() / train_count
#         validation_loss = validation_loss / test_count
#         val_acc = val_correct.double() / test_count

#         # Log to TensorBoard
#         writer.add_scalar('Train Loss', train_loss, e)
#         writer.add_scalar('Train Accuracy', train_acc, e)
#         writer.add_scalar('Validation Loss', validation_loss, e)
#         writer.add_scalar('Validation Accuracy', val_acc, e)

#         print(f"Epoch {e+1}/{epochs}: "
#               f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc*100:.2f}%, "
#               f"Validation Loss: {validation_loss:.4f}, Validation Accuracy: {val_acc*100:.2f}%")

#         if val_acc > best_accuracy:
#             torch.save(model.state_dict(), 'best_checkpoint_resnet50.model')
#             best_accuracy = val_acc

In [26]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
def train(epochs, train_loader, val_loader, criterion, optimizer, device, writer, model):
    best_accuracy = 0
    train_losses, val_losses = [], []  # For plotting learning curves
    train_accuracies, val_accuracies = [], []  # For plotting learning curves

    print("======================== Start Training ======================")
    for e in range(epochs):
        model.train()  # Set model to training mode
        train_loss = 0
        validation_loss = 0
        train_correct = 0
        val_correct = 0
        train_count = len(train_loader.dataset)
        val_count = len(val_loader.dataset)

        # Train the model
        for data, labels in train_loader:
            data, labels = data.to(device), labels.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, preds = torch.max(output, 1)
            train_correct += torch.sum(preds == labels.data)

        # Validate the model
        model.eval()  # Set model to evaluation mode
        val_labels_list = []
        val_preds_list = []
        with torch.no_grad():  # Disable gradient calculation
            for data, labels in val_loader:
                data, labels = data.to(device), labels.to(device)
                val_outputs = model(data)
                val_loss = criterion(val_outputs, labels)
                validation_loss += val_loss.item()
                _, val_preds = torch.max(val_outputs, 1)
                val_correct += torch.sum(val_preds == labels.data)
                
                # Collect the true labels and predictions for the confusion matrix
                val_labels_list.append(labels.cpu().numpy())  # Move to CPU and convert to numpy
                val_preds_list.append(val_preds.cpu().numpy())  # Move to CPU and convert to numpy

        # Compute average losses and accuracies
        train_loss = train_loss / train_count
        train_acc = train_correct.double() / train_count
        validation_loss = validation_loss / val_count
        val_acc = val_correct.double() / val_count

        # Log to TensorBoard
        writer.add_scalar('Train Loss', train_loss, e)
        writer.add_scalar('Train Accuracy', train_acc, e)
        writer.add_scalar('Validation Loss', validation_loss, e)
        writer.add_scalar('Validation Accuracy', val_acc, e)

        print(f"Epoch {e+1}/{epochs}: "
              f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc*100:.2f}%, "
              f"Validation Loss: {validation_loss:.4f}, Validation Accuracy: {val_acc*100:.2f}%")

        # Store values for learning curve plotting
        train_losses.append(train_loss)
        val_losses.append(validation_loss)
        train_accuracies.append(train_acc)
        val_accuracies.append(val_acc)

        # Compute and plot confusion matrix after each epoch
        val_labels = np.concatenate(val_labels_list)
        val_preds = np.concatenate(val_preds_list)
        cm = confusion_matrix(val_labels, val_preds, labels=np.arange(len(np.unique(val_labels))))
        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=train_loader.dataset.class_names)
        disp.plot(cmap='Blues')
        plt.title(f'Epoch {e+1} Confusion Matrix')
        # plt.savefig(f'confusion_matrix_epoch_{e+1}.png')  # Save the confusion matrix image
        plt.close()  # Close the figure to avoid overlap in future plots

        # Save the best model based on validation accuracy
        if val_acc > best_accuracy:
            torch.save(model.state_dict(), 'best_checkpoint_model.pth')
            best_accuracy = val_acc
        # break 
        
    # Save the final model after training
    torch.save(model.state_dict(), 'final_model.pth')
    
    plt.figure(figsize=(10, 5))
    plt.plot(range(epochs), train_losses, label='Train Loss', color='blue')
    plt.plot(range(epochs), val_losses, label='Validation Loss', color='red')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Learning Curve - Loss')
    plt.legend()
    plt.savefig('learning_curve_loss.png')
    plt.show()

In [ ]:
import numpy as np 
train(num_epochs, train_loader, test_loader, criterion, optimizer, device, writer, model)

======================== Start Training ======================
Epoch 1/100: Train Loss: 0.0651, Train Accuracy: 15.45%, Validation Loss: 0.0666, Validation Accuracy: 24.59%
Epoch 2/100: Train Loss: 0.0569, Train Accuracy: 29.06%, Validation Loss: 0.0597, Validation Accuracy: 25.41%
Epoch 3/100: Train Loss: 0.0517, Train Accuracy: 42.67%, Validation Loss: 0.0534, Validation Accuracy: 40.16%
Epoch 4/100: Train Loss: 0.0468, Train Accuracy: 50.79%, Validation Loss: 0.0514, Validation Accuracy: 40.98%
Epoch 5/100: Train Loss: 0.0453, Train Accuracy: 51.05%, Validation Loss: 0.0496, Validation Accuracy: 48.36%
Epoch 6/100: Train Loss: 0.0442, Train Accuracy: 52.09%, Validation Loss: 0.0487, Validation Accuracy: 45.08%
Epoch 7/100: Train Loss: 0.0419, Train Accuracy: 54.97%, Validation Loss: 0.0475, Validation Accuracy: 47.54%
Epoch 8/100: Train Loss: 0.0386, Train Accuracy: 63.35%, Validation Loss: 0.0473, Validation Accuracy: 51.64%
Epoch 9/100: Train Loss: 0.0382, Train Accuracy: 61.52%, 

In [ ]:
import os
from torch.utils.data import Dataset
from PIL import Image
from torchvision import transforms

class TestDataset(Dataset):
    def __init__(self, test_path, transform=None):
        """
        Args:
            test_path (str): Path to the directory containing images.
            transform (callable, optional): A function/transform to apply to the images.
        """
        self.test_path = test_path
        self.transform = transform
        # Get the list of image filenames in the directory (considering jpg, jpeg, png)
        self.image_files = [f for f in os.listdir(test_path) if f.endswith(('.jpg', '.jpeg', '.png'))]
        # print(self.image_files)

    def __len__(self):
        # Return the number of images in the directory
        return len(self.image_files)

    def __getitem__(self, idx):
        # Load the image
        img_name = os.path.join(self.test_path, self.image_files[idx])
        image = Image.open(img_name).convert('RGB')  # Ensure it's in RGB mode
        if self.transform:
            image = self.transform(image)
        
        # Return the image and a placeholder for labels (None or empty tuple)
        return image

# Example usage:
# Apply the necessary transformations (e.g., resizing and normalization)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet normalization
])

# Assuming the test images are stored in the 'Dataset/ChildData/Croped/Test/' folder
test_dataset = TestDataset(test_path='Dataset/ChildData/TIF_DB/', transform=transform)

# You can now use a DataLoader to load the test dataset
from torch.utils.data import DataLoader
test_loader = DataLoader(test_dataset, batch_size=131, shuffle=False)

# The test_loader can now be used to evaluate your model


In [ ]:
test_dataset[0].shape

In [ ]:
next(iter(test_loader)).shape

In [ ]:
import torch
from torchvision import datasets, transforms
from PIL import Image

def evaluate_and_print_results(test_loader, model, device, class_names):
    model.eval()  # Set model to evaluation mode

    # Iterate through the test_loader to run inference
    with torch.no_grad():  # No need to calculate gradients for inference
        for data in test_loader:  # Ignore labels (which are None)
            # Get the image filenames from the test dataset
            image_filenames = test_loader.dataset.image_files

            data = data.to(device)  # Move data to the appropriate device (e.g., 'cuda' or 'cpu')
            outputs = model(data)  # Forward pass to get predictions

            # Get the predicted class for each image in the batch
            _, predicted_classes = torch.max(outputs, 1)

            # Print out the image filenames and the corresponding predicted classes
            for idx, filename in enumerate(image_filenames):
                predicted_class = predicted_classes[idx].item()  # Get the predicted class index
                print(f"Image: {filename}, Predicted Class: {class_names[predicted_class]}")

# Example usage:
# Assuming you have a test_loader, a trained model, and the class names
class_names = ['Angry', 'Disgust', 'Fear', 'Happiness', 'Neutral', 'Sad', 'Surprised']  # Example class names
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Make sure to use the appropriate device

# Run the function to print results
evaluate_and_print_results(test_loader, model, device, class_names)


In [ ]:
#GRAD CAM

In [ ]:
import os
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

class TestDataset(Dataset):
    def __init__(self, test_path, transform=None):
        """
        Args:
            test_path (str): Path to the directory containing subfolders of images.
            transform (callable, optional): A function/transform to apply to the images.
        """
        self.test_path = test_path
        self.transform = transform
        
        # Get the list of image filenames and labels from subfolders
        self.image_files = []
        self.labels = []
        self.class_names = os.listdir(test_path)  # Subfolder names represent the class labels
        self.class_to_idx = {class_name: idx for idx, class_name in enumerate(self.class_names)}
        
        for class_name in self.class_names:
            class_folder = os.path.join(test_path, class_name)
            if os.path.isdir(class_folder):
                for img_name in os.listdir(class_folder):
                    if img_name.endswith(('.jpg', '.jpeg', '.png')):
                        self.image_files.append(os.path.join(class_folder, img_name))
                        self.labels.append(self.class_to_idx[class_name])
                        
    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        # Load the image
        img_name = self.image_files[idx]
        image = Image.open(img_name).convert('RGB')  # Ensure it's in RGB mode
        
        if self.transform:
            image = self.transform(image)
        
        label = self.labels[idx]
        return image, label


# Apply the necessary transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet normalization
])

# Assuming the test images are stored in the 'Dataset/ChildData/OUTPUTFOLDER/' folder
test_dataset = TestDataset(test_path='Dataset/ChildData/Croped/Test/', transform=transform)

# Create DataLoader for the test dataset
test_loader = DataLoader(test_dataset, batch_size=131, shuffle=False)

In [ ]:
test_dataset[0][0].shape

In [ ]:
# next(iter(test_loader))

In [ ]:
# Evaluate the model and calculate accuracy and confusion matrix
def evaluate_and_print_results(test_loader, model, device, class_names):
    model.eval()  # Set model to evaluation mode

    all_preds = []
    all_labels = []

    # Iterate through the test_loader to run inference
    with torch.no_grad():
        for data, labels in test_loader:
            data = data.to(device)  # Move data to the appropriate device (e.g., 'cuda' or 'cpu')
            labels = labels.to(device)

            outputs = model(data)  # Forward pass to get predictions
            _, predicted_classes = torch.max(outputs, 1)

            all_preds.extend(predicted_classes.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Calculate accuracy
    accuracy = np.mean(np.array(all_preds) == np.array(all_labels))
    print(f"Accuracy: {accuracy * 100:.2f}%")

    # Generate confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]  # Normalize by row (true label)

    # Plot confusion matrix
    plt.figure(figsize=(10, 7))
    sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.title('Normalized Confusion Matrix')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.show()

# Example usage:
class_names = ['Angry', 'Disgust', 'Fear', 'Happiness', 'Neutral', 'Sad', 'Surprised']  # Example class names
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use GPU if available, else CPU

model.load_state_dict(torch.load('final_model.pth'))
# Run the function to print results
evaluate_and_print_results(test_loader, model, device, class_names)